In [110]:
from sqlalchemy import create_engine
import pymysql

sqlEngine       = create_engine('mysql+pymysql://root:@127.0.0.1/kaggle_hour_price', pool_recycle=3600)
dbConnection    = sqlEngine.connect()

In [247]:
import os
os.chdir(path='/Users/liuyifu/Project_data')

In [111]:
%run package_import.ipynb

In [122]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn import preprocessing
from sklearn.metrics import classification_report, mean_absolute_error
from sklearn import linear_model

In [113]:
df_combined = pd.read_sql_query(

''' 
select *
from transformed_combined
'''
,dbConnection)

In [114]:
df_profile = pd.read_sql_query(

''' 
select *
from raw_data_profile
'''
,dbConnection)

df_profile

,column_names,col_desc,value_desc,data_type,unique_count,min_value,max_value,mean_value,median_value,null_values,total_rows,%_null_values,num_lower_outliers,num_higher_outliers
0,Id,None,nan,int64,2919,1.0,2919.0,1460.000000,1460.0,0,2919,0.000000,0.0,0.0
1,MSSubClass,Identifies the type of dwelling involved in the sale.,"{'20': '1-STORY 1946 & NEWER ALL STYLES', '30': '1-STORY 1945 & OLDER', '40': '1-STORY W/FINISHED ATTIC ALL AGES', '45': '1-1/2 STORY - UNFINISHED ALL AGES', '50': '1-1/2 STORY FINISHED ALL AGES', '60': '2-STORY 1946 & NEWER', '70': '2-STORY 1945 & OLDER', '75': '2-1/2 STORY ALL AGES', '80': 'SPLIT OR MULTI-LEVEL', '85': 'SPLIT FOYER', '90': 'DUPLEX - ALL STYLES AND AGES', '120': '1-STORY PUD (Planned Unit Development) - 1946 & NEWER', '150': '1-1/2 STORY PUD - ALL AGES', '160': '2-STORY PUD - 1946 & NEWER', '180': 'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER', '190': '2 FAMILY CONVERSION - ALL STYLES AND AGES'}",int64,16,20.0,190.0,57.137718,50.0,0,2919,0.000000,0.0,207.0
2,MSZoning,Identifies the general zoning classification of the sale.,"{'A': 'Agriculture', 'C': 'Commercial', 'FV': 'Floating Village Residential', 'I': 'Industrial', 'RH': 'Residential High Density', 'RL': 'Residential Low Density', 'RP': 'Residential Low Density Park', 'RM': 'Residential Medium Density'}",object,6,NaN,NaN,NaN,NaN,4,2919,0.001370,NaN,NaN
3,LotFrontage,Linear feet of street connected to property,{},float64,129,21.0,313.0,69.305795,68.0,486,2919,0.166495,104.0,94.0
4,LotArea,Lot size in square feet,{},int64,1951,1300.0,215245.0,10168.114080,9453.0,0,2919,0.000000,1.0,127.0
5,Street,Type of road access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved'}",object,2,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
6,Alley,Type of alley access to property,"{'Grvl': 'Gravel', 'Pave': 'Paved', 'NA': 'No alley access'}",object,3,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
7,LotShape,General shape of property,"{'Reg': 'Regular', 'IR1': 'Slightly irregular', 'IR2': 'Moderately Irregular', 'IR3': 'Irregular'}",object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
8,LandContour,Flatness of the property,"{'Lvl': 'Near Flat/Level', 'Bnk': 'Banked - Quick and significant rise from street grade to building', 'HLS': 'Hillside - Significant slope from side to side', 'Low': 'Depression'}",object,4,NaN,NaN,NaN,NaN,0,2919,0.000000,NaN,NaN
9,Utilities,Type of utilities available,"{'AllPub': 'All public Utilities (E,G,W,& S)', 'NoSewr': 'Electricity, Gas, and Water (Septic Tank)', 'NoSeWa': 'Electricity and Gas Only', 'ELO': 'Electricity only'}",object,3,NaN,NaN,NaN,NaN,2,2919,0.000685,NaN,NaN


### Read train data

In [199]:
df_combined = pd.read_sql_query(

''' 
select *
from transformed_combined
'''
,dbConnection)

# df_combined.drop(columns=['source'], inplace=True)


### Prepare data for ML

In [384]:
def preprocessing_data(input: pd.DataFrame) -> np.array:
    """[summary]

    Args:
        input (pd.DataFrame): [description]

    Returns:
        np.array: [description]
    """
    
    train_rows = input[input['source']=='train'].shape[0]
    
    cols_selected = [
            'LotArea', 'LandContour', 'Utilities',
            'LandSlope', 'BldgType', 'OverallQual',
            'OverallCond', 'ExterQual', 'ExterCond', 
            'MoSold','YearBuilt', 'SaleType', 
            'PoolArea', 'GarageArea', 'Heating', 'CentralAir', 
            'Alley', 'Street', 'TotalBsmtSF',
             'SalePrice'        
                    ]
    input = input[cols_selected]
    X = input.iloc[:, :-1]
    y = input.iloc[:, -1]

    X = pd.get_dummies(X, dummy_na=False)
    X.drop(columns=['LandSlope_Gtl', 'ExterQual_TA', 'ExterCond_Gd',
                    'SaleType_WD', 'Utilities_AllPub' ,'CentralAir_N', 'Heating_GasA', 
                    'Alley_No alley access'], inplace=True)

    X_train = X.iloc[:train_rows,:]
    X_test = X.iloc[train_rows:,:]

    y_train = y.iloc[:train_rows,]
    y_test = y.iloc[train_rows:,]
    
    return X_train, y_train, X_test, y_test


In [385]:
X_train, y_train, X_test, y_test = preprocessing_data(df_combined)

In [375]:
def color_highlight_highly_corr(val):

    threshold = 0.7

    if abs(val) >= threshold:
        color = 'red'
    else:
        color = 'black'

    return 'color: %s' % color

X_train.corr().style.applymap(color_highlight_highly_corr)

,LotArea,OverallQual,OverallCond,MoSold,YearBuilt,PoolArea,GarageArea,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSeWa,LandSlope_Mod,LandSlope_Sev,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterCond_Ex,ExterCond_Fa,ExterCond_Po,ExterCond_TA,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,Heating_Floor,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Alley_Grvl,Alley_No alley access,Alley_Pave,Street_Grvl,Street_Pave
LotArea,1.000000,0.105806,-0.005636,0.001205,0.014228,0.077672,0.180403,0.028095,0.089605,0.359475,-0.256843,0.010123,0.092649,0.540380,0.163554,0.077767,-0.015346,-0.141863,-0.167717,0.076372,-0.021622,0.003151,0.001447,-0.028664,-0.013557,0.011682,-0.016019,-0.007818,-0.002872,-0.006018,0.001076,-0.015040,0.020039,-0.005722,-0.010206,0.127918,-0.016126,-0.011477,-0.012311,0.049755,-0.029901,0.075624,-0.077747,0.197131,-0.197131
OverallQual,0.105806,1.000000,-0.091932,0.070815,0.572323,0.065166,0.562022,-0.083523,0.103611,-0.046563,0.017672,-0.001881,-0.049135,-0.043725,0.035872,-0.123989,-0.158156,-0.003723,0.128662,0.384495,-0.164663,0.547731,0.040502,-0.150923,-0.058690,0.131245,-0.065259,0.034147,0.037524,-0.037305,0.004269,-0.021172,0.327412,-0.057962,-0.058690,-0.039453,-0.091051,0.024129,-0.098545,0.272038,-0.125218,0.067704,0.038768,-0.058823,0.058823
OverallCond,-0.005636,-0.091932,1.000000,-0.003511,-0.375983,-0.001985,-0.151521,0.062886,-0.002597,-0.034591,-0.022931,0.009994,0.022696,-0.009697,0.154637,0.022057,-0.112683,-0.017261,-0.136749,-0.022983,-0.114071,-0.174598,0.139695,-0.139661,-0.084144,-0.260800,-0.028186,0.031788,-0.019156,-0.064332,0.001299,-0.019779,-0.156175,-0.050663,0.009994,-0.013147,-0.053725,-0.019156,-0.074226,0.118969,0.115899,-0.072953,-0.020830,-0.042848,0.042848
MoSold,0.001205,0.070815,-0.003511,1.000000,0.012398,-0.033737,0.027974,-0.036519,0.070935,0.030087,-0.033515,-0.051552,0.000093,0.010292,0.028213,-0.054460,0.037276,-0.007259,-0.030886,0.003091,-0.027323,0.044001,0.022577,-0.046220,-0.012805,0.015724,0.010729,0.003454,-0.011263,0.016522,0.040735,-0.054700,0.094991,0.028174,0.006568,0.041811,-0.048624,0.022996,-0.020788,0.009846,-0.004314,0.018130,-0.021780,-0.003690,0.003690
YearBuilt,0.014228,0.572323,-0.375983,0.012398,1.000000,0.004950,0.478954,-0.215457,0.049723,-0.006969,0.118321,-0.011505,-0.061090,-0.041169,-0.119096,-0.206036,-0.026063,0.086878,0.238477,0.185085,-0.119127,0.557910,-0.073043,-0.175908,-0.027113,0.255334,-0.065414,-0.028675,0.036472,-0.043304,0.002975,-0.023431,0.346954,0.012122,-0.019309,-0.169567,-0.100124,-0.094171,-0.039959,0.381831,-0.321379,0.253115,-0.016612,-0.021137,0.021137
PoolArea,0.077672,0.065166,-0.001985,-0.033737,0.004950,1.000000,0.061047,0.025697,-0.012935,-0.010922,-0.003883,-0.001798,-0.014828,-0.006511,0.030467,-0.010117,-0.013201,-0.011966,-0.019991,0.030967,-0.006759,0.016581,-0.003117,-0.009605,-0.001798,-0.039556,-0.011966,-0.003600,-0.002544,-0.005410,-0.004027,-0.004027,0.008838,-0.003117,-0.001798,-0.007675,-0.004768,-0.002544,-0.003600,0.018122,-0.012935,0.017710,-0.011676,-0.004413,0.004413
GarageArea,0.180403,0.562022,-0.151521,0.027974,0.478954,0.061047,1.000000,-0.102019,0.056776,0.050601,0.008448,0.006372,0.007136,-0.000742,0.096031,-0.085838,-0.041672,-0.095950,0.002714,0.271768,-0.093315,0.370531,-0.012663,-0.157005,-0.057936,0.121954,0.000736,-0.038068,0.012220,-0.002572,-0.005535,-0.041904,0.296671,-0.080601,-0.022781,-0.004898,-0.122424,-0.047651,-0.026480,0.230741,-0.094767,0.083793,-0.018296,0.047794,-0.047794
LandContour_Bnk,0.028095,-0.083523,0.062886,-0.036519,-0.215457,0.025697,-0.102019,1.000000,-0.039990,-0.033765,-0.629913,-0.005560,0.199310,0.015754,0.003239,0.109016,0.031941,-0.036993,-0.061802,-0.004435,0.152049,-0.129033,-0.009636,0.019460,-0.005560,-0.013590,-0.017056,-0.011131,-0.007865,0.026341,-0.012449,0.045

In [386]:
X_train, X_validate, y_train, y_validate = train_test_split(
    X_train, y_train, test_size=0.2, random_state=5   
)

In [387]:
linear_regressor = linear_model.LinearRegression()
linear_regressor.fit(X_train, y_train)
y_test_pred = linear_regressor.predict(X_validate)
mean_absolute_error(y_test_pred, y_validate)

26316.81033857103

#### Predict test data

In [265]:
y_test_pred = linear_regressor.predict(X_test)

In [248]:
df_submission = pd.read_csv('House_Prices/sample_submission.csv')

In [266]:
pd.DataFrame(y_test_pred)

,0
0,136803.666247
1,171057.033366
2,148656.517176
3,179232.067117
4,221950.003511
5,175603.632632
6,176127.784817
7,175524.960618
8,207065.197148
9,101801.049290


In [249]:
df_submission.merge()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
5,1466,177150.989247
6,1467,172070.659229
7,1468,175110.956520
8,1469,162011.698832
9,1470,160726.247831
